In [1]:
import numpy as np
import torch
import json
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from itertools import combinations
from tqdm import tqdm

In [2]:
dst_mean_thres = 40.71012391942026
dst_range_thres = 2.3691151695520034
mse_thres = 1.6497723586327953
lr = LinearRegression()
linear_model = LinearRegression()

In [3]:
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    d = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return d

In [4]:
def calcMSE(obj_arr):
    x = obj_arr[:,0].reshape(-1, 1)
    y_true = obj_arr[:,1]
    lr.fit(x, y_true)
    y_pred = lr.predict(x)
    mse = mean_squared_error(y_true, y_pred)
    return mse, lr

In [5]:
def cutScore(prediction, score_thres = 0.4):
    for pred in prediction:
        pred['boxes'] = pred['boxes'][pred['scores'] > score_thres, :]
        pred['labels'] = pred['labels'][pred['scores'] > score_thres]
        pred['scores'] = pred['scores'][pred['scores'] > score_thres]
    return prediction

In [6]:
def getCoords(boxes):
    object_coords = []
    boxes = boxes.cpu().numpy()
    box_count = 0
    for box in boxes:
        coord = [(box[0] + box[2]) / 2 - .5, (box[1] + box[3]) / 2 - .5]
        object_coords.append(coord)
        box_count = box_count + 1
        if box_count == 30:
            break
    return len(object_coords), object_coords

In [7]:
def getIds(i):
    sq = i // 5 + 1
    fr = (i + 1) - (sq - 1) * 5
    return sq, fr

In [8]:
def getAllPoints(prediction):
    all_points = {}
    points = {}
    for i in range(len(prediction)):
        sq, fr = getIds(i)
        _, p = getCoords(prediction[i]['boxes'])
        points[fr] = p
        if fr == 5:
            all_points[sq] = points.copy()
            points.clear()
    return all_points

In [9]:
def getAllScores(prediction):
    all_scores = {}
    scores = {}
    for i in range(len(prediction)):
        sq, fr = getIds(i)
        scores[fr] = [torch.IntTensor.item(s) for s in prediction[i]['scores']]
        if fr == 5:
            all_scores[sq] = scores.copy()
            scores.clear()
    return all_scores

In [10]:
def estimatePoint(fr_iter, obj_arr, fr_to_look, linear_model):
    linear_model.fit(np.array(fr_iter).reshape(-1,1), obj_arr[:,0].reshape(-1,1))
    return linear_model.predict(np.array([fr_to_look]).reshape(-1,1))

In [11]:
def isMonotonic(A): 
    return (all(A[i] <= A[i + 1] for i in range(len(A) - 1)) or
            all(A[i] >= A[i + 1] for i in range(len(A) - 1))) 

In [12]:
def sortDict(d, reverse=False):
    d_sorted = {}
    for s in sorted(d, reverse=reverse):
        d_sorted[s] = d[s]
    return d_sorted

In [13]:
def sortPointsByScores(val, score):
    objects = {}
    for fr_iter in combinations(range(1, 6), 3):
        fr1, fr2, fr3 = fr_iter
        for n1, p1 in enumerate(val[fr1]):
            for n2, p2 in enumerate(val[fr2]):
                dst1 = distance(p2, p1) / (fr2 - fr1)
                if dst1 > dst_mean_thres: continue
                for n3, p3 in enumerate(val[fr3]):
                    dst2 = distance(p3, p2) / (fr3 - fr2)
                    if dst2 > dst_mean_thres: continue
                    if np.abs(dst2 - dst1) > dst_range_thres: continue
                    obj_arr = np.array([p1, p2, p3])
                    x_monotonic = isMonotonic(obj_arr[:,0])
                    y_monotonic = isMonotonic(obj_arr[:,1])
                    if x_monotonic or y_monotonic:
                        s = (score[fr1][n1] + score[fr2][n2] + score[fr3][n3]) / 3.
                        objects[s] = {fr1:p1, fr2:p2, fr3:p3}
                        objects = sortDict(objects, reverse=True)
    return objects

In [14]:
def checkPointsBoundary(fr_iter, obj_arr, linear_model):
    exist1 = False
    exist2 = False
    temp = {}
    for fr_to_look in range(1, 6):
        if fr_to_look not in fr_iter:
            x = estimatePoint(fr_iter, obj_arr, fr_to_look, linear_model)
            y = lr.predict(x)
            temp[fr_to_look] = [x[0][0], y[0]]
            if x[0][0] >= -0.5 and x[0][0] <= 639.5 and y[0] >= -0.5 and y[0] <= 479.5:
                if exist1 == False:
                    exist1 = True
                else:
                    exist2 = True
    return exist1 and exist2, temp

In [15]:
def getLinearPoints(points_linear, fr_iter, obj_arr, temp, val, overlap_thres):
    fr1, fr2, fr3 = fr_iter
    p1, p2, p3 = obj_arr.tolist()
    points_linear[fr1].append(p1)
    points_linear[fr2].append(p2)
    points_linear[fr3].append(p3)
    for fr_to_look, new_point in temp.items():
        min_dst = overlap_thres
        min_p = None
        for num, p in enumerate(val[fr_to_look]):
            if distance(new_point, p) < min_dst:
                min_dst = distance(new_point, p)
                min_p = p
        if min_p != None:
            points_linear[fr_to_look].append(min_p)
        else:
            points_linear[fr_to_look].append(new_point)

In [16]:
def getAllLinearPoints(objects, val, overlap_thres=1.5):
    points_linear = {}
    for i in range(1, 6):
        points_linear[i] = []                
    
    for objs in objects.values():
        fr_iter = objs.keys()
        fr1, fr2, fr3 = fr_iter
        fr_iter = (fr1, fr2, fr3)
        p1, p2, p3 = objs.values()
        if p1 in points_linear[fr1]: continue
        if p2 in points_linear[fr2]: continue
        if p3 in points_linear[fr3]: continue
        obj_arr = np.array([p1, p2, p3])
        mse, lr = calcMSE(obj_arr)
        if mse < mse_thres:
            two_other_points_exist, estimation = checkPointsBoundary(fr_iter, obj_arr, linear_model)
            if two_other_points_exist:
                getLinearPoints(points_linear, fr_iter, obj_arr, estimation, val, overlap_thres)
            
    return points_linear

In [17]:
def assertObjNumPerFrame(all_points_linear):
    incomplete_key = []
    for key, val in all_points_linear.items():
        length = []
        for fr in range(1, 6):
            length.append(len(val[fr]))
        if max(length) - min(length) > 0:
            incomplete_key.append(key)
    assert(len(incomplete_key) == 0)

In [18]:
def makeSubmission(all_points_linear, score_thres):
    submission=[]
    for sq in all_points_linear.keys():
        for fr in range(1, 6):
            num = len(all_points_linear[sq][fr])
            submission.append({"sequence_id" : sq, 
                               "frame" : fr, 
                               "num_objects" : num, 
                               "object_coords" : all_points_linear[sq][fr]})

    with open('best_submission/best_submission_clean_{:.2f}-st.json'.format(score_thres), 'w') as outfile:
        json.dump(submission, outfile)

In [19]:
%%time
for st in np.arange(0.1, 1, 0.1):
    print('Prediction for {:.2f} score threshold on progress...'.format(st))
    prediction = torch.load('prediction_fasterrcnn-resnet50-fpn_6-epochs.pt')
    prediction = cutScore(prediction, score_thres=st)
    all_points = getAllPoints(prediction)
    all_scores = getAllScores(prediction)

    all_points_linear = {}
    for key in tqdm(range(1, 5121)):
        val = all_points[key]
        score = all_scores[key]
        objects = sortPointsByScores(val, score)
        points_linear = getAllLinearPoints(objects, val, overlap_thres=1.5)
        all_points_linear[key] = points_linear

    assertObjNumPerFrame(all_points_linear)
    makeSubmission(all_points_linear, st)
print('DONE!')

Prediction for 0.10 score threshold on progress...


100%|█████████████████████████████████████████████████████████████████████████████| 5120/5120 [00:31<00:00, 163.95it/s]


Prediction for 0.20 score threshold on progress...


100%|█████████████████████████████████████████████████████████████████████████████| 5120/5120 [00:21<00:00, 239.48it/s]


Prediction for 0.30 score threshold on progress...


...

Prediction for 0.80 score threshold on progress...


100%|█████████████████████████████████████████████████████████████████████████████| 5120/5120 [00:15<00:00, 335.76it/s]


Prediction for 0.90 score threshold on progress...


100%|█████████████████████████████████████████████████████████████████████████████| 5120/5120 [00:14<00:00, 353.11it/s]


DONE!
Wall time: 9min 10s
